In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
path = '../data/weatherdata'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

weather_df = pd.concat(li, axis=0, ignore_index=True)

/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,44,48,53,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (45,46,50,55,59,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,56,57,64,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danielchoy/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,43,44,45,49,54,58,59) have mi

In [58]:
pd.reset_option('display.max_columns', None)

Convert wind speed from meters/second to miles/hour. The conversion can be done using the following formula:

$$s(mph) = \frac{s(m/s) \times 3600}{1000 \times 1.6}$$

https://www.weather.gov/pqr/wind

In [3]:
def to_milesperhour(num):
    convert = (num * 3600)/(1000 * 1.6)
    return convert

In [4]:
def clean_weatherdata(df):
    nyweather= df.copy()
    nyweather = nyweather[['DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'WND', 'TMP',
             'DEW', 'SLP', 'AA1','AA2']]
    nyweather['DATE'] = nyweather['DATE'].str.replace('T', ' ')
    nyweather['TMP'] = nyweather['TMP'].astype(str)
    nyweather['TMP'] = nyweather['TMP'].str.slice(0, -2)
    nyweather['TMP'] = nyweather['TMP'].str.replace('+','')
    nyweather['TMP'] = nyweather['TMP'].astype(int)
    nyweather['TMP'] = nyweather['TMP']/10
    
    print('complete part 1')
    
    nyweather['WND'] = nyweather['WND'].astype(str)
    nyweather = pd.concat([nyweather, nyweather.WND.str.split(',', expand = True)],1)
    nyweather = nyweather.rename(columns={3:'wind_speed'})
    nyweather['wind_speed'] = nyweather['wind_speed'].astype(int)
    nyweather['wind_speed'] = nyweather['wind_speed']/10
    nyweather['wind_speed'] = nyweather['wind_speed'].map(to_milesperhour)
    nyweather['windy'] = nyweather['wind_speed'].apply(lambda x: 'calm' if x < 4 else 'breeze' if x < 12 else \
                                        'moderate breeze' if x < 24 else 'strong breeze' if x < 31 else \
                                        'gale' if x < 63 else 'storm force')
    return nyweather

In [8]:
weather_df = clean_weatherdata(weather_df)

complete part 1


In [9]:
weather_df

,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,WND,TMP,DEW,SLP,AA1,AA2,0,1,2,wind_speed,4,windy
0,2017-01-01 00:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,1,N,0077,1",6.7,"-0028,1","10143,1",NaN,NaN,200,1,N,17.325,1,moderate breeze
1,2017-01-01 00:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","220,5,N,0051,5",6.7,"-0039,5","10139,5","01,0000,9,5",NaN,220,5,N,11.475,5,breeze
2,2017-01-01 01:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","210,5,N,0051,5",6.7,"-0044,5","10131,5","01,0000,9,5",NaN,210,5,N,11.475,5,breeze
3,2017-01-01 02:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,5,N,0051,5",6.7,"-0044,5","10128,5","01,0000,9,5",NaN,200,5,N,11.475,5,breeze
4,2017-01-01 03:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","200,1,N,0051,1",6.7,"-0044,1","10128,1",NaN,NaN,200,1,N,11.475,1,breeze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109768,2019-12-31 20:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,5,N,0046,5",7.2,"+0011,5","10028,5","01,0000,9,5",NaN,240,5,N,10.350,5,breeze
109769,2019-12-31 21:00:00,4,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,1,N,0046,1",7.2,"+0011,1","10028,1",NaN,NaN,240,1,N,10.350,1,breeze
109770,2019-12-31 21:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","250,5,N,0062,5",7.2,"+0000,5","10033,5","01,0000,9,5",NaN,250,5,N,13.950,5,moderate breeze
109771,2019-12-31 22:51:00,7,40.77944,-73.88035,3.4,"LAGUARDIA AIRPORT, NY US","240,5,N,0046,5",7.2,"+0006,5","10036,5","01,0000,9,5",NaN,240,5,N,10.350,5,breeze


In [ ]:
weather_df['AA1']